In [6]:
path = "logs_yelp-b264-f8-GPUV100"
elapsed_time = []
with open(path, "r") as f:
  for line in f:
    if "[" in line:
      with open("yelp-b264-f8-GPUV100", "a") as f2:
        f2.write(line)
      # [0] Elapsed time: 99.98m - Train time: 93.42m - Test time: 6.56
      el_time = float(line.split(" ")[3].split("m")[0])
      elapsed_time.append(el_time)

In [21]:
gpu_time = sum(elapsed_time)/len(elapsed_time)
cpu_time = 142.77
improvement = (cpu_time-gpu_time)/cpu_time*100
print('improvement on time', improvement)
print(f'Total time cpu for 100 epochs {cpu_time*100:.2f} -  min {cpu_time*100/60:.2f} hours')
print(f'Total time gpu for 100 epochs {gpu_time*100:.2f} -  min {gpu_time*100/60:.2f} hours')
print(f'Improved time {cpu_time*100-gpu_time*100:.2f} -  min {(cpu_time*100-gpu_time*100)/60:.2f} hours')


improvement on time 25.12852840232542
Total time cpu for 100 epochs 14277.00 -  min 237.95 hours
Total time gpu for 100 epochs 10689.40 -  min 178.16 hours
Improved time 3587.60 -  min 59.79 hours


In [10]:
abs((cpu_time - gpu_time))/gpu_time

0.33562220517522035

In [1]:
import os

cpu_cores = os.cpu_count()
print("Number of CPU cores available:", cpu_cores)

Number of CPU cores available: 12


In [35]:
import torch


device = torch.device("cpu: ")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
local_hr = 0.5
local_rr = 0.5
local_ndcg = 0.5

local_metrics = torch.tensor([local_hr, local_rr, local_ndcg], dtype=torch.float32)
# Gather metrics on rank 0
gathered_metrics = [torch.zeros_like(local_metrics) for _ in range(4)]

In [42]:
local_metrics

tensor([0.5000, 0.5000, 0.5000])

In [48]:
gathered_metrics = [local_metrics, local_metrics*2, local_metrics*3, local_metrics*4]

In [51]:
gathered_metrics = [
    torch.tensor([0.1, 0.2, 0.1]),
    torch.tensor([0.2, 0.3, 0.2]),
    torch.tensor([0.15, 0.25, 0.15]),
    torch.tensor([0.05, 0.15, 0.05])
]
gathered_metrics

[tensor([0.1000, 0.2000, 0.1000]),
 tensor([0.2000, 0.3000, 0.2000]),
 tensor([0.1500, 0.2500, 0.1500]),
 tensor([0.0500, 0.1500, 0.0500])]

In [53]:
metrics_tensor = torch.stack(gathered_metrics)
metrics_tensor

tensor([[0.1000, 0.2000, 0.1000],
        [0.2000, 0.3000, 0.2000],
        [0.1500, 0.2500, 0.1500],
        [0.0500, 0.1500, 0.0500]])

In [56]:
average_metrics = torch.mean(metrics_tensor, dim=0)
average_metrics

tensor([0.1250, 0.2250, 0.1250])

In [59]:
print(f"Average HR: {average_metrics[0]:.4f}")

Average HR: 0.1250


In [55]:
print(f"Average HR: {average_metrics[0]:.4f}")
print(f"Average MRR: {average_metrics[1]:.4f}")
print(f"Average NDCG: {average_metrics[2]:.4f}")

Average HR: 0.1250
Average MRR: 0.2250
Average NDCG: 0.1250


In [5]:
from time import time

s1 = time()
for i in range(10000000):
  pass
s2 = time()
print(s2-s1)

0.268662691116333


In [66]:
d = dict()
# access last elemento on dictionry
bool(d.items())


False